In [1]:
from __future__ import division
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
import os

In [2]:
df = pd.read_csv('result.csv')

In [3]:
df.rename(columns ={'Unnamed: 0': 'new column name'}, inplace = True)

In [4]:
df.drop(["new column name"], axis = 1, inplace = True)

In [5]:
df.head()

,Date,sales,debit,credit,customer_name
0,2011-04-01,Opening Balance,5940.0,0.0,"COMMUNITY HEALTH CENTRE, PALLUR"
1,2011-04-04,Cash,0.0,5940.0,"COMMUNITY HEALTH CENTRE, PALLUR"
2,2011-04-01,Opening Balance,86484.0,0.0,"DENTSALES,KANNUR."
3,2011-04-04,Cash,0.0,19500.0,"DENTSALES,KANNUR."
4,2011-04-15,Cash,0.0,19500.0,"DENTSALES,KANNUR."


In [6]:
df['gross_total'] = df['debit'] - df['credit']

In [7]:
df['balance'] = df.groupby('customer_name')['gross_total'].cumsum()

In [8]:
df.head()

,Date,sales,debit,credit,customer_name,gross_total,balance
0,2011-04-01,Opening Balance,5940.0,0.0,"COMMUNITY HEALTH CENTRE, PALLUR",5940.0,5940.0
1,2011-04-04,Cash,0.0,5940.0,"COMMUNITY HEALTH CENTRE, PALLUR",-5940.0,0.0
2,2011-04-01,Opening Balance,86484.0,0.0,"DENTSALES,KANNUR.",86484.0,86484.0
3,2011-04-04,Cash,0.0,19500.0,"DENTSALES,KANNUR.",-19500.0,66984.0
4,2011-04-15,Cash,0.0,19500.0,"DENTSALES,KANNUR.",-19500.0,47484.0


In [9]:
df.drop(["gross_total"], axis = 1, inplace = True)

In [10]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

In [11]:
df.head()

,Date,sales,debit,credit,customer_name,balance
0,2011-04-01,Opening Balance,5940.0,0.0,"COMMUNITY HEALTH CENTRE, PALLUR",5940.0
1,2011-04-04,Cash,0.0,5940.0,"COMMUNITY HEALTH CENTRE, PALLUR",0.0
2,2011-04-01,Opening Balance,86484.0,0.0,"DENTSALES,KANNUR.",86484.0
3,2011-04-04,Cash,0.0,19500.0,"DENTSALES,KANNUR.",66984.0
4,2011-04-15,Cash,0.0,19500.0,"DENTSALES,KANNUR.",47484.0


In [12]:
df['days'] = -(df.groupby('customer_name')['Date'].diff(periods=-1))

In [13]:
df.head()

,Date,sales,debit,credit,customer_name,balance,days
0,2011-04-01,Opening Balance,5940.0,0.0,"COMMUNITY HEALTH CENTRE, PALLUR",5940.0,3 days
1,2011-04-04,Cash,0.0,5940.0,"COMMUNITY HEALTH CENTRE, PALLUR",0.0,NaT
2,2011-04-01,Opening Balance,86484.0,0.0,"DENTSALES,KANNUR.",86484.0,3 days
3,2011-04-04,Cash,0.0,19500.0,"DENTSALES,KANNUR.",66984.0,11 days
4,2011-04-15,Cash,0.0,19500.0,"DENTSALES,KANNUR.",47484.0,12 days


In [14]:
df['days'] = df['days'].apply(lambda x: x.days)

In [15]:
df.head()

,Date,sales,debit,credit,customer_name,balance,days
0,2011-04-01,Opening Balance,5940.0,0.0,"COMMUNITY HEALTH CENTRE, PALLUR",5940.0,3.0
1,2011-04-04,Cash,0.0,5940.0,"COMMUNITY HEALTH CENTRE, PALLUR",0.0,NaN
2,2011-04-01,Opening Balance,86484.0,0.0,"DENTSALES,KANNUR.",86484.0,3.0
3,2011-04-04,Cash,0.0,19500.0,"DENTSALES,KANNUR.",66984.0,11.0
4,2011-04-15,Cash,0.0,19500.0,"DENTSALES,KANNUR.",47484.0,12.0


In [16]:
df = df.fillna(0)

In [17]:
df['CEP'] = df['balance']/df['days']

In [18]:
df.replace([np.inf, -np.inf], np.nan, inplace=True) 

In [19]:
df = df.fillna(0)

In [20]:
df.head()

,Date,sales,debit,credit,customer_name,balance,days,CEP
0,2011-04-01,Opening Balance,5940.0,0.0,"COMMUNITY HEALTH CENTRE, PALLUR",5940.0,3.0,1980.000000
1,2011-04-04,Cash,0.0,5940.0,"COMMUNITY HEALTH CENTRE, PALLUR",0.0,0.0,0.000000
2,2011-04-01,Opening Balance,86484.0,0.0,"DENTSALES,KANNUR.",86484.0,3.0,28828.000000
3,2011-04-04,Cash,0.0,19500.0,"DENTSALES,KANNUR.",66984.0,11.0,6089.454545
4,2011-04-15,Cash,0.0,19500.0,"DENTSALES,KANNUR.",47484.0,12.0,3957.000000


## Clusters based on debit

In [21]:
df_debit = df.groupby('customer_name')['debit'].sum()

In [22]:
df_debit.head()

customer_name
13TH MID-TERM CONFERENCE, KANNUIR SALES      20920.0
32 CARROT(Dr)                                47609.0
38 TH ISPPD - SALES                         267425.0
38 TH ISPPD SALES-1                         248590.0
47TH KERALA STATE DENTAL CONFERENCE        2437937.0
Name: debit, dtype: float64

In [23]:
df_debit.to_csv('df_debit.csv')

In [24]:
df_debit = pd.read_csv('df_debit.csv')

In [25]:
#df_debit = pd.read_csv('df_debit.csv', names=["customer_name", "debit"],skipinitialspace=True)

In [26]:
df_debit.head()

,customer_name,debit
0,"13TH MID-TERM CONFERENCE, KANNUIR SALES",20920.0
1,32 CARROT(Dr),47609.0
2,38 TH ISPPD - SALES,267425.0
3,38 TH ISPPD SALES-1,248590.0
4,47TH KERALA STATE DENTAL CONFERENCE,2437937.0


In [27]:
kmeans = KMeans(n_clusters=8)
kmeans.fit(df_debit[['debit']])
df_debit['debitCluster'] = kmeans.predict(df_debit[['debit']])

In [28]:
def order_cluster(cluster_field_name, target_field_name,df_,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df_.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df_,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [29]:
df_debit = order_cluster('debitCluster', 'debit',df_debit,False)

In [30]:
df_debit.groupby('debitCluster')['debit'].describe()

,count,mean,std,min,25%,50%,75%,max
debitCluster,,,,,,,,
0,1.0,1.073816e+08,NaN,1.073816e+08,1.073816e+08,1.073816e+08,1.073816e+08,1.073816e+08
1,1.0,4.604789e+07,NaN,4.604789e+07,4.604789e+07,4.604789e+07,4.604789e+07,4.604789e+07
2,3.0,2.886534e+07,1.880302e+06,2.671039e+07,2.821183e+07,2.971327e+07,2.994281e+07,3.017235e+07
3,4.0,2.077240e+07,3.238951e+06,1.636706e+07,1.935953e+07,2.158512e+07,2.299798e+07,2.355232e+07
4,19.0,6.670493e+06,1.628752e+06,4.862924e+06,5.384733e+06,6.100149e+06,7.745134e+06,1.096499e+07
5,80.0,2.684461e+06,7.984174e+05,1.743712e+06,2.072902e+06,2.443529e+06,3.103727e+06,4.623641e+06
6,565.0,7.451794e+05,2.980050e+05,4.038550e+05,5.141140e+05,6.458570e+05,9.048210e+05,1.703784e+06
7,5722.0,6.090656e+04,8.590422e+04,0.000000e+00,5.053250e+03,2.117450e+04,7.790950e+04,4.015400e+05


## CEP clustering

In [31]:
df_cep = df.groupby('customer_name')['CEP'].mean()

In [32]:
df_cep.shape

(6395,)

In [33]:
df_cep.to_csv('df_cep.csv')

In [34]:
df_cep = pd.read_csv('df_cep.csv')

In [35]:
#df_cep = pd.read_csv('df_cep.csv', names=["customer_name", "cep"],skipinitialspace=True)

In [36]:
df_cep.head()

,customer_name,CEP
0,"13TH MID-TERM CONFERENCE, KANNUIR SALES",-4175.111111
1,32 CARROT(Dr),872.180853
2,38 TH ISPPD - SALES,-11252.222222
3,38 TH ISPPD SALES-1,-22531.363636
4,47TH KERALA STATE DENTAL CONFERENCE,-25872.725806


In [37]:
kmeans = KMeans(n_clusters=8)
kmeans.fit(df_cep[['CEP']])
df_cep['CEPCluster'] = kmeans.predict(df_cep[['CEP']])

In [38]:
df_cep = order_cluster('CEPCluster', 'CEP',df_cep,False)

In [39]:
df_cep.groupby('CEPCluster')['CEP'].describe()

,count,mean,std,min,25%,50%,75%,max
CEPCluster,,,,,,,,
0,1.0,8.600240e+06,NaN,8.600240e+06,8.600240e+06,8.600240e+06,8.600240e+06,8.600240e+06
1,2.0,2.009735e+06,190885.270635,1.874759e+06,1.942247e+06,2.009735e+06,2.077223e+06,2.144712e+06
2,3.0,9.068350e+05,54993.963310,8.435768e+05,8.886157e+05,9.336546e+05,9.384642e+05,9.432737e+05
3,5.0,5.043917e+05,124331.960101,3.580617e+05,4.081243e+05,5.185219e+05,5.693529e+05,6.678978e+05
4,30.0,1.627862e+05,47442.506748,1.122945e+05,1.232979e+05,1.487531e+05,1.851298e+05,2.746332e+05
5,123.0,5.664787e+04,16510.540152,3.678872e+04,4.406514e+04,5.309797e+04,6.572550e+04,1.075768e+05
6,538.0,1.672433e+04,7017.448592,8.641541e+03,1.101844e+04,1.461556e+04,2.080509e+04,3.597116e+04
7,5693.0,9.422292e+02,4122.595346,-1.400000e+05,2.305556e+01,3.247559e+02,1.676644e+03,8.632168e+03


In [40]:
df_clusters=df_debit

In [41]:
df_clusters.head()

,customer_name,debit,debitCluster
0,"13TH MID-TERM CONFERENCE, KANNUIR SALES",20920.0,7
1,32 CARROT(Dr),47609.0,7
2,38 TH ISPPD - SALES,267425.0,7
3,38 TH ISPPD SALES-1,248590.0,7
4,A R LAB SYSTEM ERANJIPALAM,10812.0,7


In [42]:
df_clusters['CEP'] = df_cep['CEP']

In [43]:
df_clusters['CEPCluster'] = df_cep['CEPCluster']

In [44]:
df_clusters.head()

,customer_name,debit,debitCluster,CEP,CEPCluster
0,"13TH MID-TERM CONFERENCE, KANNUIR SALES",20920.0,7,-4175.111111,7
1,32 CARROT(Dr),47609.0,7,872.180853,7
2,38 TH ISPPD - SALES,267425.0,7,-11252.222222,7
3,38 TH ISPPD SALES-1,248590.0,7,-22531.363636,7
4,A R LAB SYSTEM ERANJIPALAM,10812.0,7,-25872.725806,7


## Recency

In [45]:
df_purchase = df[df.debit != 0]

In [46]:
df_purchase.head()

,Date,sales,debit,credit,customer_name,balance,days,CEP
0,2011-04-01,Opening Balance,5940.0,0.0,"COMMUNITY HEALTH CENTRE, PALLUR",5940.0,3.0,1980.0
2,2011-04-01,Opening Balance,86484.0,0.0,"DENTSALES,KANNUR.",86484.0,3.0,28828.0
5,2011-04-27,Sales 4%,12675.0,0.0,"DENTSALES,KANNUR.",60159.0,3.0,20053.0
6,2011-04-30,Sales 4%,7830.0,0.0,"DENTSALES,KANNUR.",67989.0,0.0,0.0
8,2011-05-11,Sales 4%,8650.0,0.0,"DENTSALES,KANNUR.",57139.0,5.0,11427.8


In [47]:
df_max_purchase = df_purchase.groupby('customer_name').Date.max().reset_index()

In [48]:
df_max_purchase.columns = ['customer_name','MaxPurchaseDate']

In [49]:
df_max_purchase['Recency'] = (df_max_purchase['MaxPurchaseDate'].max() - df_max_purchase['MaxPurchaseDate']).dt.days

In [50]:
df_clusters = pd.merge(df_clusters, df_max_purchase[['customer_name','Recency']], on='customer_name')

In [51]:
df_clusters.head()

,customer_name,debit,debitCluster,CEP,CEPCluster,Recency
0,"13TH MID-TERM CONFERENCE, KANNUIR SALES",20920.0,7,-4175.111111,7,645
1,32 CARROT(Dr),47609.0,7,872.180853,7,12
2,38 TH ISPPD - SALES,267425.0,7,-11252.222222,7,1466
3,38 TH ISPPD SALES-1,248590.0,7,-22531.363636,7,1461
4,A R LAB SYSTEM ERANJIPALAM,10812.0,7,-25872.725806,7,976


In [52]:
kmeans = KMeans(n_clusters=8)
kmeans.fit(df_clusters[['Recency']])
df_clusters['RecencyCluster'] = kmeans.predict(df_clusters[['Recency']])

In [53]:
df_clusters = order_cluster('RecencyCluster', 'Recency',df_clusters,False)

In [54]:
df_clusters.groupby('RecencyCluster')['Recency'].describe()

,count,mean,std,min,25%,50%,75%,max
RecencyCluster,,,,,,,,
0,409.0,3221.621027,150.845885,2997.0,3121.0,3135.0,3358.00,3501.0
1,1031.0,2767.258972,96.036269,2515.0,2755.0,2780.0,2807.50,2993.0
2,394.0,2252.705584,152.285805,1992.0,2109.0,2245.5,2406.00,2510.0
3,599.0,1727.275459,105.090163,1542.0,1670.0,1674.0,1816.00,1990.0
4,550.0,1354.134545,96.225336,1157.0,1309.0,1325.0,1430.75,1541.0
5,701.0,957.226819,133.863304,719.0,841.0,962.0,1088.00,1151.0
6,707.0,479.886846,111.551503,305.0,396.0,455.0,579.00,718.0
7,1992.0,126.913153,93.728821,0.0,25.0,139.0,213.00,303.0


## Frequency

In [55]:
df_frequency = df_purchase.groupby('customer_name').Date.count().reset_index() #here total purchases are considedred intead of voucher_number based purchases

In [56]:
df_frequency.columns = ['customer_name','Frequency']

In [57]:
df_clusters = pd.merge(df_clusters, df_frequency, on='customer_name')

In [58]:
df_clusters.shape

(6383, 8)

In [59]:
kmeans = KMeans(n_clusters=8)
kmeans.fit(df_clusters[['Frequency']])
df_clusters['FrequencyCluster'] = kmeans.predict(df_clusters[['Frequency']])

In [60]:
df_clusters = order_cluster('FrequencyCluster', 'Frequency',df_clusters,False)

In [61]:
df_clusters.groupby('FrequencyCluster')['Frequency'].describe()

,count,mean,std,min,25%,50%,75%,max
FrequencyCluster,,,,,,,,
0,1.0,4116.000000,NaN,4116.0,4116.0,4116.0,4116.00,4116.0
1,1.0,1263.000000,NaN,1263.0,1263.0,1263.0,1263.00,1263.0
2,20.0,581.450000,76.997932,484.0,527.5,553.0,612.75,764.0
3,63.0,368.063492,50.858714,296.0,324.5,354.0,410.50,465.0
4,147.0,218.836735,37.343654,166.0,183.5,215.0,246.50,291.0
5,430.0,111.769767,22.866197,80.0,92.0,108.0,129.75,165.0
6,983.0,47.461851,14.984049,27.0,35.0,44.0,60.00,79.0
7,4738.0,6.109329,6.493064,1.0,1.0,3.0,9.00,26.0


In [67]:
df_clusters

,customer_name,debit,debitCluster,CEP,CEPCluster,Recency,RecencyCluster,Frequency,FrequencyCluster
0,"13TH MID-TERM CONFERENCE, KANNUIR SALES",2.092000e+04,0,-4.175111e+03,0,645,6,2,0
1,ABDUL RAZAK.PP,1.905000e+04,0,5.406000e+03,0,579,6,6,0
2,AFIA DENTAL CLINIC. KOTTAKKAL,1.015600e+05,0,-1.166667e+03,0,396,6,17,0
3,AGNEL JOSE,1.764500e+04,0,2.293079e+03,0,651,6,4,0
4,AL ANSAR DENTAL CLINIC KODUVALLY,1.387600e+05,0,4.899134e+02,0,575,6,10,0
...,...,...,...,...,...,...,...,...,...
6378,"DR.RAVINDRANATH, TALIPARAMBA",5.684379e+06,3,2.061621e+05,3,8,7,664,5
6379,KCK EQUIPMENTS CO. PVT. LTD. (DR),2.281321e+07,4,3.580617e+05,4,1310,4,529,5
6380,AKSHARA,4.282028e+05,1,9.684900e+03,1,961,5,547,5
6381,COUNTER SALE,4.862924e+06,3,1.853159e+05,3,0,7,4116,7


In [63]:
df_clusters['debitCluster']=df_clusters['debitCluster'].replace({0:7,1:6,2:5,3:4,4:3,5:2,6:1,7:0})

In [64]:
df_clusters['CEPCluster']=df_clusters['CEPCluster'].replace({0:7,1:6,2:5,3:4,4:3,5:2,6:1,7:0})

In [65]:
df_clusters['FrequencyCluster']=df_clusters['FrequencyCluster'].replace({0:7,1:6,2:5,3:4,4:3,5:2,6:1,7:0})

## Overall Score

In [68]:
df_clusters['overall_score'] = df_clusters['RecencyCluster'] + df_clusters['FrequencyCluster'] + df_clusters['CEPCluster'] +df_clusters['debitCluster']

In [70]:
df_clusters.head()

,customer_name,debit,debitCluster,CEP,CEPCluster,Recency,RecencyCluster,Frequency,FrequencyCluster,overall_score
0,"13TH MID-TERM CONFERENCE, KANNUIR SALES",20920.0,0,-4175.111111,0,645,6,2,0,6
1,ABDUL RAZAK.PP,19050.0,0,5406.000000,0,579,6,6,0,6
2,AFIA DENTAL CLINIC. KOTTAKKAL,101560.0,0,-1166.666667,0,396,6,17,0,6
3,AGNEL JOSE,17645.0,0,2293.079279,0,651,6,4,0,6
4,AL ANSAR DENTAL CLINIC KODUVALLY,138760.0,0,489.913396,0,575,6,10,0,6


In [71]:
df_clusters['segment'] = 'Low-Value'
df_clusters.loc[df_clusters['overall_score']>12,'segment'] = 'Mid-Value' 
df_clusters.loc[df_clusters['overall_score']>16,'segment'] = 'High-Value' 

In [72]:
df_clusters.head()

,customer_name,debit,debitCluster,CEP,CEPCluster,Recency,RecencyCluster,Frequency,FrequencyCluster,overall_score,segment
0,"13TH MID-TERM CONFERENCE, KANNUIR SALES",20920.0,0,-4175.111111,0,645,6,2,0,6,Low-Value
1,ABDUL RAZAK.PP,19050.0,0,5406.000000,0,579,6,6,0,6,Low-Value
2,AFIA DENTAL CLINIC. KOTTAKKAL,101560.0,0,-1166.666667,0,396,6,17,0,6,Low-Value
3,AGNEL JOSE,17645.0,0,2293.079279,0,651,6,4,0,6,Low-Value
4,AL ANSAR DENTAL CLINIC KODUVALLY,138760.0,0,489.913396,0,575,6,10,0,6,Low-Value


In [73]:
df_clusters.to_csv('credit_risk.csv')

## END

In [66]:
end

NameError: name 'end' is not defined

In [ ]:
df_debit.head()

In [ ]:
df_count = df_debit['debitCluster'].value_counts()

In [ ]:
df_count.head()

In [ ]:
df_count.to_csv('debit_count.csv')

In [ ]:
df_count = pd.read_csv('debit_count.csv',names=['cluster','count'],skipinitialspace=True)

In [ ]:
df_count.head()

In [ ]:
k=[]
for value in df_count['count']:
    k.append(value)


In [ ]:
i=0
while(i<=4):
    if(df_count['count'][i]==max(k)):
        c = df_count['cluster'][i]
        break
    else:
        i = i+1

In [ ]:
c

In [ ]:
s = df_count['count'].sum()

In [ ]:
p = max(df_debit['debitCluster'].value_counts())

In [ ]:
df_clusters = df_debit[df_debit['debitCluster']!=c]

In [ ]:
df_clusters.groupby('debitCluster')['debit'].describe()

In [ ]:
if c==0:
    df_clusters['debitCluster']=df_clusters['debitCluster'].replace({1:4,2:5,3:6})
elif c==1:
    df_clusters['debitCluster']=df_clusters['debitCluster'].replace({2:5,3:6})
elif c==2:
    df_clusters['debitCluster']=df_clusters['debitCluster'].replace({3:6})

In [ ]:
df_clusters.groupby('debitCluster')['debit'].describe()

In [ ]:
df_debit = order_cluster('debitCluster', 'debit',df_debit,False)

In [ ]:
if p<0.75*s:
    df_clusters = df_debit
else:
    df_clusters = df_debit[df_debit['debitCluster']!=c]
    if c==0:
        df_clusters['debitCluster']=df_clusters['debitCluster'].replace({1:4,2:5,3:6})
    elif c==1:
        df_clusters['debitCluster']=df_clusters['debitCluster'].replace({2:5,3:6})
    elif c==2:
        df_clusters['debitCluster']=df_clusters['debitCluster'].replace({3:6})
    df_debitc = df_debit[df_debit['debitCluster']==c]
    kmeans = KMeans(n_clusters=4)
    kmeans.fit(df_debitc[['debit']])
    df_debitc['debitClusterc'] = kmeans.predict(df_debitc[['debit']])
    if c==1:
        df_debitc['debitCluster']=df_debitc["debitClusterc"].replace({0:1,1:2,2:3,3:4})
    elif c==2:
        df_debitc['debitClusterc']=df_debitc["debitClusterc"].replace({0:2,1:3,2:4,3:5})
    elif c==3:
        df_debitc['debitClusterc']=df_debitc["debitClusterc"].replace({0:3,1:4,2:5,3:6})
    df_debitc.drop(['debitCluster'],axis=1)
    df_debitc.rename(columns = {'debitClusterc':'debitCluster'})
    df_clusters = pd.concat([df_clusters,df_debitc])
    

In [ ]:
df_clusters.to_csv("clusters.csv")
os.remove("debit_count.csv")
os.remove("df_debit.csv")

In [ ]:
df_clusters.head()

In [ ]:
df_clusters.groupby('debitCluster')['debit'].describe()